In [2]:
import pandas as pd
import numpy as np
import re
from collections import Counter
from konlpy.tag import Okt
import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings('ignore')

In [10]:
greed_word = ['매수','사','사다','사라','사면','사고','줍다','들어오다','들어가다','타다','수급','매집','올라타다' # 주식 구매 단어
              ,'탑승','불나방','담다'
              ,'오르다','올라가다','올리다','올려주다','올린다','오름','올려놓다','오른','상향' # 주식 가격 상승 단어
              ,'양봉','상방','상승','살아나다','양전','상한','반등','폭등','퍽등','급등'
              ,'탐욕','찬티','좋다','간다','가다','가즈','싸다','익절','제발','최고','돌파','수익','위대하다','먹다' # 탐욕 단어
              ,'기회','호재','감사','감사하다','대박','대단하다','승리','찬양','믿다','회복','갓','부활','영차'
              ,'개꿀']

fear_word = ['공매도','공매','매도','팔','파다','팔다','팔고','팔면','던지다','털다','탈출','튀다','튀어','설거지' # 주식 판매 단어
             ,'손절','버리다'
             ,'떨어지다','떨구다','빠지다','하락','폭락','떡락','조정','급락','음봉','하방','폭포수','음전' # 주식 가격 하락 단어
             ,'반토막','내리다','내려오다','깨지다','대퍽락','나락','붕괴','추락'
             ,'공포','안티','망하다','물리다','끝나다','손해','폭망','거품','무섭다','자살','악재','상폐','개미지옥' # 공포 단어
             ,'시발','염병','욕','짜증나다','걸레','어휴','개','놈','아가리','빡치다','지랄','손실','버티다','존버'
             ,'개관','주가조작','쓰레기','죽다','패닉','홀딩','바닥','흑우','추매','추미애']

In [4]:
df = pd.read_csv('./src/네이버종토방댓글_카카오_15000.csv')
df = df[df['댓글'] != 'error'] # 크롤링 에러 제거
df = df.dropna() # 결측치 제거
df['한글댓글'] = df['댓글'].str.replace('\[삭제된 게시물의 답글\]',' ')
df['한글댓글'] = df['한글댓글'].str.replace('답글:',' ')
df['한글댓글'] = df['한글댓글'].str.replace('[^가-힣]',' ').str.replace(' +',' ').str.strip()
df = df[df['한글댓글'] != '']
df = df.reset_index(drop=True)
df

,날짜,댓글,조회수,좋아요,싫어요,한글댓글
0,2022-05-03,개미들 조련하는법,26,1,0,개미들 조련하는법
1,2022-05-03,6만원,38,1,0,만원
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2,카카오는 끝물입니다 사실상
3,2022-05-03,안티 살발하다,34,2,0,안티 살발하다
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1,이런 호로개미지옥탕을 만드신
...,...,...,...,...,...,...
295849,2017-10-18,어차피 또 폭등할텐데 떨필요있나 기회주면...,795,15,3,어차피 또 폭등할텐데 떨필요있나 기회주면
295850,2017-10-18,오늘 물렸다고 생각하는 사람들은 도대체 ...,961,3,2,오늘 물렸다고 생각하는 사람들은 도대체
295851,2017-10-18,내일도,618,5,1,내일도
295852,2017-10-18,아마존 per는 300?,745,4,1,아마존 는


In [25]:
df[df['날짜'] > '2022-04-01']

,날짜,댓글,조회수,좋아요,싫어요,한글댓글
0,2022-05-03,개미들 조련하는법,26,1,0,개미들 조련하는법
1,2022-05-03,6만원,38,1,0,만원
2,2022-05-03,카카오는 끝물입니다 사실상,72,2,2,카카오는 끝물입니다 사실상
3,2022-05-03,안티 살발하다,34,2,0,안티 살발하다
4,2022-05-03,이런 호로개미지옥탕을 만드신,58,3,1,이런 호로개미지옥탕을 만드신
...,...,...,...,...,...,...
5202,2022-04-02,"[속보] 카카오 이사회서 빠진 , 두나무...",642,1,0,속보 카카오 이사회서 빠진 두나무
5203,2022-04-02,형님들 주말 시간 될 때 동의 한번 해주...,295,0,2,형님들 주말 시간 될 때 동의 한번 해주
5204,2022-04-02,억지로 주가부양중,517,3,10,억지로 주가부양중
5205,2022-04-02,답글:억지로 주가부양중,389,5,1,억지로 주가부양중


In [22]:
okt = Okt()

date = '2022-04-01'
df2 = df[df['날짜'] > date].reset_index(drop=True)

greed_dict = {}
fear_dict = {}
for i in greed_word:
    greed_dict[i] = 0
for i in fear_word:
    fear_dict[i] = 0

words = []
for i in range(df2.shape[0]):
    cmt = df2['한글댓글'][i]
    word = okt.morphs(cmt, stem=True)
    words += word
    
words_dict = dict(Counter(words))
for i in greed_word:
    try:
        greed_dict[i] = words_dict[i]
    except:
        pass
for i in fear_word:
    try:
        fear_dict[i] = words_dict[i]
    except:
        pass

In [1]:
fear_dict

NameError: name 'Counter' is not defined

In [24]:
greed_dict

{'매수': 90,
 '사': 11,
 '사다': 38,
 '사라': 20,
 '사면': 7,
 '사고': 2,
 '줍다': 8,
 '들어오다': 15,
 '들어가다': 14,
 '타다': 49,
 '수급': 3,
 '매집': 2,
 '올라타다': 0,
 '탑승': 0,
 '불나방': 0,
 '담다': 22,
 '오르다': 74,
 '올라가다': 36,
 '올리다': 36,
 '올려주다': 3,
 '올린다': 3,
 '오름': 2,
 '올려놓다': 2,
 '오른': 3,
 '상향': 0,
 '양봉': 10,
 '상방': 0,
 '상승': 23,
 '살아나다': 1,
 '양전': 1,
 '상한': 18,
 '반등': 19,
 '폭등': 32,
 '퍽등': 0,
 '급등': 9,
 '탐욕': 2,
 '찬티': 2,
 '좋다': 36,
 '간다': 46,
 '가다': 143,
 '가즈': 22,
 '싸다': 17,
 '익절': 3,
 '제발': 18,
 '최고': 28,
 '돌파': 13,
 '수익': 9,
 '위대하다': 0,
 '먹다': 29,
 '기회': 27,
 '호재': 13,
 '감사': 2,
 '감사하다': 1,
 '대박': 6,
 '대단하다': 3,
 '승리': 2,
 '찬양': 0,
 '믿다': 10,
 '회복': 3,
 '갓': 5,
 '부활': 0,
 '영차': 3,
 '개꿀': 3}